Calculate the population impacted by Pfas

计算被PFAS影响的人口

In [1]:
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
import myfunction as mf
path_data_raw = "C:/Users/dell/OneDrive/file/"
path_country_nc = "C:/Users/dell/OneDrive/file/nc"
path_country_csv = "C:/Users/dell/OneDrive/file/csv/"
path_one_spdb = 'C:/Users/dell/OneDrive/file/SPDB/'
drive_letter = 'E:'

path_pre = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/pretreatment/"
path_match = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/match/"
path_semdata = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/semdata/"


path_2_preanalysis_data = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part2_sem/preanalysis/"
path_2_preanalysis_fig = drive_letter + "/wyy/code_project/running_outcome/final_fig/SPDB/part2_sem/preanalysis/"


path_3_sw_forecast = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part3_forecast/sw_forecast/"
path_temp = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/temp/"

path_4_risk_assess = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part4_assess/"
path_4_risk_assess_grid = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part4_assess/grid/"

path_1_describe_fig_fish = drive_letter + "/wyy/code_project/running_outcome/final_fig/SPDB/part1_describe/fish_diff/"
path_1_describe_global_map = drive_letter + "/wyy/code_project/running_outcome/final_fig/SPDB/part1_describe/global_map/"

mark_num = "25"
meta_name = "meta_data.csv"
list_pfas =['PFOA', 'PFNA', 'PFDA', 'PFUnDA','PFDoDA','PFTrDA', 'PFTeDA', 'PFHxS', 'PFOS', 'FOSA', 'PFBA', 'PFPeA', 'PFHxA', 'PFHpA','PFBS']
list_pfas_lc = ['PFOA', 'PFNA', 'PFDA', 'PFUnDA','PFDoDA','PFTrDA', 'PFTeDA', 'PFHxS', 'PFOS', 'FOSA']
list_pfas_sc = ['PFBA', 'PFPeA', 'PFHxA', 'PFHpA','PFBS']
list_color = ["#4d8cbf", "#4f9c8b", "#555c6c", "#d77563", "#7d84a8", "#84aeb8", "#c3473b", "#89756d","#ffb3cc","#9a7ebf","#ffddb8", "#c4eaff", "#d1c6ff", "#c2ffbf", "#f5f5b0",]

In [4]:
import pandas as pd
import os


os.chdir(r'E:\wyy\code_project\running_outcome\final_data\SPDB\part4_assess\hi')
output_path = r'E:\wyy\code_project\running_outcome\final_data\SPDB\part4_assess\hi'


country_df = pd.read_csv('country.csv')
wiw_df = pd.read_csv('without_improved_water.csv')
pop_df = pd.read_csv('country_pop.csv')

country_map = country_df[['country_id', 'country_code', 'region']].set_index('country_code')
wiw_df = wiw_df.merge(country_map, left_on='country_code', right_index=True)
wiw_df = wiw_df[['country_id', 'year', 'wiw', 'region']]
wiw_df['wiw'] = wiw_df.groupby('country_id')['wiw'].transform(lambda x: x.fillna(x.median()))
wiw_df['wiw'] = wiw_df['wiw'].fillna(wiw_df.groupby('year')['wiw'].transform('median'))
wiw_df['wiw'] = wiw_df['wiw'].round().astype(int)
result_df = wiw_df.merge(pop_df, on=['country_id', 'year'])
result_df['per_wiw'] = (result_df['wiw'] / result_df['pop']).round(8)

def adjust_per_wiw(row, median_per_wiw):
    if row['pop'] == 0 or row['wiw'] == 0:
        return median_per_wiw
    return row['per_wiw']

# group by region and year to get the median per_wiw
for (region, year), group in result_df.groupby(['region', 'year']):
    median_per_wiw = group['per_wiw'].median()
    result_df.loc[(result_df['region'] == region) & (result_df['year'] == year), 'per_wiw'] = \
        result_df.loc[(result_df['region'] == region) & (result_df['year'] == year)].apply(
            adjust_per_wiw, axis=1, median_per_wiw=median_per_wiw)

output_file = os.path.join(output_path, "per_wiw.csv")
result_df.to_csv(output_file, index=False)

print("处理完成，结果已保存到per_wiw.csv")

处理完成，结果已保存到per_wiw.csv


In [5]:
df_per_wiw = pd.read_csv(output_file)
df_per_wiw = df_per_wiw[['year','country_id','per_wiw']]
df_grid = pd.read_csv(path_4_risk_assess_grid + 'new_grid_year.csv')
df_grid = df_grid[['lon', 'lat', 'year','country_id']]

df_grid_per_wiw = df_grid.merge(df_per_wiw, on=['country_id', 'year'])
df_grid_per_wiw = df_grid_per_wiw[['lon', 'lat', 'year','per_wiw']]
df_grid_per_wiw.to_csv(path_4_risk_assess_grid + 'per_wiw_grid_year.csv', index=False)

e:\progress\anaconda2\lib\site-packages\pandas\core\reshape\merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():


In [9]:
import pandas as pd
import numpy as np
import os

input_path = r'E:\wyy\code_project\running_outcome\final_data\SPDB\part4_assess\thq'
output_path = r'E:\wyy\code_project\running_outcome\final_data\SPDB\part4_assess\thq'
grid_path = r'E:\wyy\code_project\running_outcome\final_data\SPDB\part4_assess\grid'


if not os.path.exists(output_path):
    os.makedirs(output_path)

def calculate_statistics(df, prefix):
    stats_df = df[['lon', 'lat', 'year']].copy()
    thq_columns = df.columns[3:]
    
    stats_df[f'{prefix}_min'] = df[thq_columns].min(axis=1)
    stats_df[f'{prefix}_max'] = df[thq_columns].max(axis=1)
    stats_df[f'{prefix}_2.5%'] = df[thq_columns].quantile(0.025, axis=1)
    stats_df[f'{prefix}_50%'] = df[thq_columns].median(axis=1)
    stats_df[f'{prefix}_97.5%'] = df[thq_columns].quantile(0.975, axis=1)
    stats_df[f'{prefix}_mean'] = df[thq_columns].mean(axis=1)
    stats_df[f'{prefix}_SD'] = df[thq_columns].std(axis=1)
    
    return stats_df



In [7]:

from tqdm.notebook import tqdm
input_path = r'E:\wyy\code_project\running_outcome\final_data\SPDB\part4_assess\thq'
output_path = r'E:\wyy\code_project\running_outcome\final_data\SPDB\part4_assess\thq'
grid_path = r'E:\wyy\code_project\running_outcome\final_data\SPDB\part4_assess\grid'
per_wiw_df = pd.read_csv(os.path.join(grid_path, 'per_wiw_grid_year.csv'))
pop_df = pd.read_csv(os.path.join(grid_path, 'population_year.csv'))

pop_data = pop_df.merge(per_wiw_df, on=['lon', 'lat', 'year'], how='left')
pop_data['per_wiw'] = pop_data['per_wiw'].fillna(0)

In [ ]:

from tqdm.notebook import tqdm
import numpy as np

per_wiw_df = pd.read_csv(os.path.join(grid_path, 'per_wiw_grid_year.csv'))
pop_df = pd.read_csv(os.path.join(grid_path, 'population_year.csv'))

pop_data = pop_df.merge(per_wiw_df, on=['lon', 'lat', 'year'], how='left')
pop_data['per_wiw'] = pop_data['per_wiw'].fillna(0)

pop_dict = pop_data.set_index(['lon', 'lat', 'year']).to_dict()

lr_df = pd.read_csv(os.path.join(input_path, 'lr_merged_thq.csv'))
lr_imp_pop = lr_df.copy()
thq_columns = lr_df.columns[3:]

for col in tqdm(thq_columns, desc="Processing lr columns"):
    new_col_name = f'imp_pop_{col[9:]}' 

    mask = (lr_df[col]) >= 1
    lr_imp_pop[new_col_name] = np.where(mask, 
                                        [pop_dict['pop'].get((lon, lat, year), 0) 
                                         for lon, lat, year in zip(lr_df['lon'], lr_df['lat'], lr_df['year'])],
                                        0)
lr_imp_pop = lr_imp_pop.drop(columns=thq_columns)
lr_imp_pop.to_csv(os.path.join(output_path, 'lr_imp_pop_d.csv'), index=False)

lr_stats_pop = calculate_statistics(lr_imp_pop, 'imp_pop')
lr_stats_pop.to_csv(os.path.join(output_path, 'lr_pop_statistics_d.csv'), index=False)



sw_df = pd.read_csv(os.path.join(input_path, 'sw_merged_thq.csv'))
sw_imp_pop = sw_df.copy()
thq_columns = sw_df.columns[3:]

for col in tqdm(thq_columns, desc="Processing sw columns"):
    new_col_name = f'imp_pop_{col[10:]}'

    mask = sw_df[col] >= 1
    sw_imp_pop[new_col_name] = np.where(mask, 
                                        [pop_dict['pop'].get((lon, lat, year), 0) * pop_dict['per_wiw'].get((lon, lat, year), 0)
                                         for lon, lat, year in zip(sw_df['lon'], sw_df['lat'], sw_df['year'])],
                                        0)

sw_imp_pop = sw_imp_pop.drop(columns=thq_columns)
sw_imp_pop.to_csv(os.path.join(output_path, 'sw_imp_pop.csv'), index=False)

sw_stats_pop = calculate_statistics(sw_imp_pop, 'imp_pop')
sw_stats_pop.to_csv(os.path.join(output_path, 'sw_pop_statistics.csv'), index=False)

merge_df = pd.read_csv(os.path.join(input_path, 'merged_thq.csv'))
merge_imp_pop = merge_df.copy()
thq_columns = merge_df.columns[3:]

for col in tqdm(thq_columns, desc="Processing merge columns"):
    new_col_name = f'imp_pop_{col[10:]}'

    mask = merge_df[col] >= 1

    merge_imp_pop[new_col_name] = np.where(mask, 
                                        [pop_dict['pop'].get((lon, lat, year), 0) * pop_dict['per_wiw'].get((lon, lat, year), 0)
                                         for lon, lat, year in zip(merge_df['lon'], merge_df['lat'], merge_df['year'])],
                                        0)

merge_imp_pop = merge_imp_pop.drop(columns=thq_columns)
merge_imp_pop.to_csv(os.path.join(output_path, 'merge_imp_pop.csv'), index=False)

merge_stats_pop = calculate_statistics(merge_imp_pop, 'imp_pop')
merge_stats_pop.to_csv(os.path.join(output_path, 'merge_pop_statistics.csv'), index=False)


In [ ]:

lr_imp_pop = pd.read_csv(os.path.join(output_path, 'lr_imp_pop_d.csv'))
lr_stats_pop = calculate_statistics(lr_imp_pop, 'imp_pop')
lr_stats_pop.to_csv(os.path.join(output_path, 'lr_pop_statistics_d.csv'), index=False)

combined_imp_pop = lr_imp_pop.copy()
sw_imp_pop_cols = sw_imp_pop.columns[3:]
other_imp_pop_cols = merge_imp_pop.columns[3:]


combined_imp_pop.to_csv(os.path.join(output_path, 'combined_imp_pop.csv'), index=False)

combined_stats_pop = calculate_statistics(combined_imp_pop, 'imp_pop')
combined_stats_pop.to_csv(os.path.join(output_path, 'combined_pop_statistics.csv'), index=False)


In [8]:
def calculate_statistics(df, prefix):
    stats_df = df[['lon', 'lat', 'year']].copy()
    thq_columns = df.columns[3:]
    
    stats_df[f'{prefix}_min'] = df[thq_columns].min(axis=1)
    stats_df[f'{prefix}_max'] = df[thq_columns].max(axis=1)
    stats_df[f'{prefix}_2.5%'] = df[thq_columns].quantile(0.025, axis=1)
    stats_df[f'{prefix}_5%'] = df[thq_columns].quantile(0.05, axis=1)
    stats_df[f'{prefix}_10%'] = df[thq_columns].quantile(0.1, axis=1)
    stats_df[f'{prefix}_20%'] = df[thq_columns].quantile(0.2, axis=1)
    stats_df[f'{prefix}_50%'] = df[thq_columns].median(axis=1)
    stats_df[f'{prefix}_80%'] = df[thq_columns].quantile(0.8, axis=1)
    stats_df[f'{prefix}_90%'] = df[thq_columns].quantile(0.9, axis=1)
    stats_df[f'{prefix}_95%'] = df[thq_columns].quantile(0.95, axis=1)
    stats_df[f'{prefix}_97.5%'] = df[thq_columns].quantile(0.975, axis=1)
    stats_df[f'{prefix}_mean'] = df[thq_columns].mean(axis=1)
    stats_df[f'{prefix}_SD'] = df[thq_columns].std(axis=1)
    
    return stats_df

In [9]:
combined_stats_pop = calculate_statistics(combined_imp_pop, 'imp_pop')
combined_stats_pop.to_csv(os.path.join(output_path, 'combined_pop_statistics2.csv'), index=False)